In [8]:
from src.datasets.novelty import *

In [19]:
data.split()

In [29]:
with open('./temp.json','w') as f:
    f.writelines([json.dumps(i)+"\n" for i in d[3:10]])

In [1]:
project = "NLI"
NEPTUNE_API = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMTg3MzU5NjQtMmIxZC00Njg0LTgzYzMtN2UwYjVlYzVhNDg5In0="
import neptune
project = neptune.init(f"aparkhi/{project}", api_token=NEPTUNE_API)

In [2]:
from src.defaults import *

In [23]:
# download_models_from_neptune("NLI-81")
field = load_field('NLI-81')
# field = None


dataset_conf = {'dataset': 'dlnd', 'max_num_sent': 50,"sent_tokenizer":"spacy","batch_size":32,"device":"cuda"}
# dataset_conf = {'dataset': 'dlnd', 'max_num_sent': 50,"sent_tokenizer":"spacy", "tokenizer":'spacy',"max_len":50,"batch_size":32,"device":"cuda"}
model_conf = {'results_dir': 'results', 'device': 'cuda', 'dropout': 0.3, 'dataset': 'dlnd', 'hidden_size': 400, 'use_glove': False, "max_num_sent": 50}

In [9]:
from src.datasets.novelty import *
data = dlnd(dataset_conf,sentence_field = field)


src.datasets.novelty.DLND
.data/dlnd


In [11]:
for i in data.train_iter:
    print(i)
    break


[torchtext.data.batch.Batch of size 32]
	[.source]:[torch.cuda.LongTensor of size 32x50x50 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 32x50x50 (GPU 0)]
	[.label]:[torch.cuda.LongTensor of size 32 (GPU 0)]


In [36]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class DAN(nn.Module):
    def __init__(self,conf,encoder):
        super(DAN,self).__init__()
        self.conf = conf
        self.num_sent = conf["max_num_sent"]
        self.encoder = encoder
        self.translate = nn.Linear(2 * self.conf["encoder_dim"], self.conf["hidden_size"])
        self.template = nn.Parameter(torch.zeros((1)), requires_grad=True)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(conf["dropout"])

        self.mlp_f = nn.Linear(self.conf["hidden_size"], self.conf["hidden_size"])
        self.mlp_g = nn.Linear(2*self.conf["hidden_size"], self.conf["hidden_size"])
        self.mlp_h = nn.Linear(2*self.conf["hidden_size"], self.conf["hidden_size"])
        self.linear = nn.Linear(self.conf["hidden_size"],2)

    def encode_sent(self,inp):
        batch_size,_,_ = inp.shape
        x = inp.view(-1,self.sent_len)

        x_padded_idx = x.sum(dim=1) != 0    
        x_enc = []
        for sub_batch in x[x_padded_idx].split(64):
            x_enc.append(self.encoder(sub_batch)[0])
        x_enc = torch.cat(x_enc, dim=0)

        x_enc_t = torch.zeros((batch_size * self.num_sent, x_enc.size(1))).to(
            self.template.device
        )

        x_enc_t[x_padded_idx] = x_enc
        x_enc_t = x_enc_t.view(batch_size, self.num_sent, -1)
    
        embedded = self.dropout(self.translate(x_enc_t))
        embedded = self.act(embedded)
        embedded = embedded.permute(1, 0, 2)
        return embedded


    def forward(self,x0,x1):
        x0_enc = self.encode_sent(x0).permute(1,0,2)
        x1_enc = self.encode_sent(x1).permute(1,0,2)

        f1 = self.act(self.dropout(self.mlp_f(x0_enc)))
        f2 = self.act(self.dropout(self.mlp_f(x1_enc)))

        score1 = torch.bmm(f1, torch.transpose(f2, 1, 2))
        prob1 = F.softmax(score1.view(-1, self.num_sent)).view(-1, self.num_sent, self.num_sent)

        score2 = torch.transpose(score1.contiguous(), 1, 2)
        score2 = score2.contiguous()

        prob2 = F.softmax(score2.view(-1, self.num_sent)).view(-1, self.num_sent, self.num_sent)

        sent1_combine = torch.cat((x0_enc, torch.bmm(prob1, x1_enc)), 2)
        sent2_combine = torch.cat((x1_enc, torch.bmm(prob2, x0_enc)), 2)

        

        g1 = self.act(self.dropout(self.mlp_g(sent1_combine)))
        g2 = self.act(self.dropout(self.mlp_g(sent2_combine)))

        sent1_output = torch.sum(g1, 1)  
        sent1_output = torch.squeeze(sent1_output, 1)
    
        sent2_output = torch.sum(g2, 1)  
        sent2_output = torch.squeeze(sent2_output, 1)


        input_combine = torch.cat((sent1_output * sent2_output, torch.abs(sent1_output - sent2_output)), 1)
        
        h = self.act(self.dropout(self.mlp_h(input_combine)))
        opt = self.linear(h)
        return opt
    

In [39]:
DAN(model_conf,model)

DAN(
  (encoder): AttnBiLSTM_snli(
    (encoder): Attn_Encoder(
      (embedding): Embedding(33934, 300)
      (translate): Linear(in_features=300, out_features=400, bias=True)
      (relu): ReLU()
      (dropout): Dropout(p=0.3, inplace=False)
      (lstm_layer): LSTM(400, 400, batch_first=True, dropout=0.3, bidirectional=True)
      (attention): Attention(
        (Ws): Linear(in_features=800, out_features=200, bias=False)
        (Wa): Linear(in_features=200, out_features=1, bias=False)
      )
    )
    (fc_in): Linear(in_features=3200, out_features=400, bias=True)
    (fcs): ModuleList(
      (0): Linear(in_features=400, out_features=400, bias=True)
    )
    (fc_out): Linear(in_features=400, out_features=3, bias=True)
    (relu): ReLU()
    (softmax): Softmax(dim=2)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (translate): Linear(in_features=800, out_features=400, bias=True)
  (act): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (mlp_f): Linear(in_features=400, out_fe

In [27]:
def load_encoder(_id):
    model_path = os.path.join('./results/',_id,"model.pt")
    model_data = torch.load(model_path)
    return model_data

In [28]:
model_data = load_encoder('NLI-81')

In [31]:
from src.model.nli_models import *

In [33]:
model = attn_bilstm_snli(model_data['options'])

In [38]:
model_conf["encoder_dim"] = model_data["options"]["hidden_size"]

In [41]:
def load_encoder1(enc_data):
    if enc_data['options'].get("attention_layer_param",0)==0:
        model = bilstm_snli(enc_data["options"])
    elif enc_data['options'].get("r",0)==0:
        model = attn_bilstm_snli(enc_data["options"])
    else:
        model = struc_attn_snli(enc_data["options"])
    return model
    

In [40]:
model_data

{'accuracy': 81.19284698232066,
 'dataset_conf': {'batch_size': 128,
  'dataset': 'snli',
  'device': 'cuda',
  'eos_token': '<eos>',
  'init_token': '<sos>',
  'lower': True,
  'max_len': 50,
  'max_word_len': 10,
  'pad_token': '<pad>',
  'preprocessing': None,
  'tokenize': 'spacy',
  'tokenizer': 'spacy',
  'unk_token': '<unk>',
  'use_char_emb': False,
  'use_vocab': True},
 'model_dict': OrderedDict([('encoder.embedding.weight',
               tensor([[0., 0., 0.,  ..., 0., 0., 0.],
                       [0., 0., 0.,  ..., 0., 0., 0.],
                       [0., 0., 0.,  ..., 0., 0., 0.],
                       ...,
                       [0., 0., 0.,  ..., 0., 0., 0.],
                       [0., 0., 0.,  ..., 0., 0., 0.],
                       [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')),
              ('encoder.translate.weight',
               tensor([[-0.1381,  0.0271, -0.0114,  ..., -0.1154, -0.1232, -0.0280],
                       [-0.1849,  0.0321, -0.0733,  ...